# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [26]:
import pandas as pd

## imports for Python, Pandas

In [27]:
import json
from pandas.io.json import json_normalize
import numpy as np



Path and File of downloaded json world bank projects file

In [28]:
PATH = 'C:\\Users\\User\\Documents\\PAUL\\Springboard\\core\\'
FILE = 'world_bank_projects.json'

JSON_F = PATH + FILE

Read in JSON file to be processed

In [29]:
# load json as string
json.load((open(JSON_F)))
# load as Pandas dataframe
dfjson = pd.read_json(JSON_F)
print(dfjson.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [30]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [31]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [32]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [33]:
# calculate top 10 projects by country
dfprojects = pd.DataFrame(dfjson.countryshortname)
dfprojects['numprojects'] = 0
dfprojects = dfprojects.groupby('countryshortname').count().nlargest(10,'numprojects')
print(dfprojects)

                    numprojects
countryshortname               
China                        19
Indonesia                    19
Vietnam                      17
India                        16
Yemen, Republic of           13
Bangladesh                   12
Morocco                      12
Nepal                        12
Africa                       11
Mozambique                   11


In [34]:
# convert mjtheme_namecode to a data frame and calculate top 10 major themes
countryl = []
codel = []
namel = []
jsontheme = dfjson[['countryshortname','mjtheme_namecode']]

for i,j in jsontheme.iterrows():
    for ddict in j.mjtheme_namecode:
        countryl.append(j.countryshortname)
        codel.append(ddict['code'])
        namel.append(ddict['name'])

# create a data frame with countryshortname, code and name.
# countryshortname is optional and not needed for this excersize.
themedict = {'countryshortname':countryl,'code':codel,'name':namel}
dftheme = pd.DataFrame(themedict)
dftheme['numthemes'] = 0
dftopthemes = dftheme.groupby('name').count().nlargest(10,'numthemes')
print(dftopthemes[['numthemes']])

                                              numthemes
name                                                   
Environment and natural resources management        223
Rural development                                   202
Human development                                   197
Public sector governance                            184
Social protection and risk management               158
Financial and private sector development            130
                                                    122
Social dev/gender/inclusion                         119
Trade and integration                                72
Urban development                                    47


In [35]:
# fix missing names given a code in mjtheme_namecode and recalculate top 10 themes
dftu = dftheme[['code','name']].drop_duplicates().replace('',np.NaN).dropna()
dictcodemap = dict(zip(dftu.code,dftu.name))
dftheme['name'] = dftheme['code'].map(dictcodemap)
dftopthemes = dftheme.groupby('name').count().nlargest(10,'numthemes')
print(dftopthemes[['numthemes']])

                                              numthemes
name                                                   
Environment and natural resources management        250
Rural development                                   216
Human development                                   210
Public sector governance                            199
Social protection and risk management               168
Financial and private sector development            146
Social dev/gender/inclusion                         130
Trade and integration                                77
Urban development                                    50
Economic management                                  38
